# 1. Vector Space Model of Word Meaning

The goal of this problem set is to make you familiar with vector space model of word meaning. You may reuse some of functions you coded in Assignment 1. 

### Warning: This assignment may take substantial time to run if you are not optimizing your code. Make sure you have plenty of time to run if you are new to programming.

Go to https://drive.google.com/drive/folders/1Pe6D713L9S0GWwb_XzeLXAeNZOrBqZaN?usp=sharing and click add shortcut to drive. This will add the data required for this problem set to your Google drive.

<img src="https://drive.google.com/uc?id=1LqHisiziX8Ri94Xs6Cv8mhx6vivFM3kS" alt="Drawing" height="300"/>

Caution: Since this is real language on Twitter and deals with current events, some of it could be disturbing. In the later section of the course, we will deal with ethics and what is appropriate and what is not. 



Run the below code snippet. It will generate a URL which generates an authorization code.* Enter it below to give Colab access to your Google drive. 

*Copy function may not work. If so, manually copy the authorization code.

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


When you run the `ls` command below, you should see the files in the tweets folder.

> Indented block






In [3]:
!ls "/content/drive/My Drive/tweets"

20000_tweets.jsonl
20000_tweets.txt
covid-tweets-2020-08-10-2020-08-21.tokenized.txt
covid-tweets-2020-08-10-2020-08-21.trigrams.txt
GoogleNews-vectors-negative300.bin.gz
stop_words.txt


In [4]:
# let's read tweets. These tweets are already tokenized and cleaned (Assignment 1)
tweets = open("/content/drive/My Drive/tweets/covid-tweets-2020-08-10-2020-08-21.tokenized.txt", "r").read().split("\n")
tweets = [tweet.split() for tweet in tweets]

print(len(tweets))


312878


## Problem 1.1: Word space model

Compute the most important context words of `ventilator`. Use Pointwise Mutual Information (PMI) to rank the context words (Refer to Lecture 4).

We define context as up to 3 words to the left and 3 words to the right. Ignore stop words and words that do not start with [a-z#]. Also ignore words that are not in the top 1000 frequent words.

These context words define the dimensions of the vector space model. Represent each word as a vector (dictionary/counter) of context words with PMI as the importance of the context word. Print the top 20 context words for each.

This is the sample output I got for `ventilator`. Your numbers need not match mine but the ranked list should look close to what I have.

```
[('put', 18.280538283196606), ('wearing', 17.587373569812726), ('even', 17.58651933524197), ('like', 17.402738298715878), ('covid', 17.172590097063086), ('patients', 16.894419647496004), ('use', 16.894298589380956), ('die', 16.89426559608771), ('days', 16.89415252713107), ('needed', 16.489137134110106), ('month', 16.48907033839664), ('weeks', 16.488913820220848), ('away', 16.48879303327717), ('week', 16.488739054051933), ('person', 16.488678720881293), ('good', 16.488160838026904), ('deaths', 16.487822204799755), ('go', 16.487564042558112), ('would', 16.48707075078768), ('one', 16.48706217686235), ('get', 16.486565870239033)]
```



Let's first load stop words.

In [5]:
stop_words = set()
def load_stop_words():
  words = open("/content/drive/My Drive/tweets/stop_words.txt", "r").read().split("\n")
  for word in words:
    stop_words.add(word.strip())

load_stop_words()
print(len(stop_words))
print(stop_words)

134
{"'ll", 'both', 'those', 'a', 'whom', 'his', 'theirs', 'off', 'of', 'when', 'most', 'i', 'each', 'myself', 'that', 'while', 'more', 'because', 'with', 'we', 'any', 'same', 'here', 'why', 'her', 'URL', 'from', 'than', 'yourselves', 'as', 'he', 'only', 'me', 'will', 'this', 'hers', 'my', 'them', 'against', 'very', '@USER', 'and', 'yourself', 'is', 'all', 'few', 'through', 'which', 'after', 'other', 'the', 'to', 'how', "n't", 'ourselves', 'themselves', 'their', 'not', 'once', 's', 'don', 'again', 'was', 'into', 'himself', 'our', 'such', 'some', 'about', 'herself', 'over', 'doing', 'by', 'who', 'now', 'should', 'during', "'ve", 'out', 'or', 'can', 'below', 'where', 'you', 'it', 'itself', 'between', 'does', 'before', 'what', 'being', 'nor', 'so', 'just', 'did', 'they', 'do', 'if', 'were', 'ours', 'these', 'there', 'she', 'on', 'under', 'having', 'up', 'no', 'yours', 'have', 'him', 'an', 'has', 'at', 't', 'am', 'own', 'but', 'amp', 'for', "'ld", 'until', 'down', 'been', 'your', 'too', 'b



```
# This is formatted as code
```

Let's build the word vectors

In [42]:
# help here: https://www.kaggle.com/gabrielaltay/word-vectors-from-pmi-matrix
from collections import Counter
from itertools import combinations
from math import log
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pformat
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds, norm
from nltk.util import skipgrams
from string import punctuation
from scipy import sparse
from scipy.sparse import linalg

unigram_counts = Counter()
skipgram_counts = Counter()
x2i, i2x = {}, {}
pmi_samples = Counter()

tweets_trimmed = [[token for token in tweet if token not in stop_words] for tweet in tweets]



def compute_unigrams():
  for idx, tweet in enumerate(tweets_trimmed):
    for token in tweet:
      unigram_counts[token] += 1
    if idx % 50000 == 0:
        print(f'finished {idx/len(tweets):.2%} of tweets')

  print('Vocab size: {}'.format(len(unigram_counts)))
  print('Most common: {}'.format(unigram_counts.most_common(10)))



def compute_skip_grams():
  for idx, tweet in enumerate(tweets_trimmed):
      skip_gram = list(skipgrams(tweet, 2, 2))
      for context in skip_gram:
        skipgram_counts[context] += 1
      if idx % 50000 == 0:
        print(f'finished {idx/len(tweets_trimmed):.2%} of tweets')


  print('Skipgram size: {}'.format(len(skipgram_counts)))
  print('Most common skipgram: {}'.format(skipgram_counts.most_common(10)))


# def remove_freq_infreq_unigrams():
#   print('unigrms before: {}'.format(len(unigram_counts)))
#   min_count = (1 / 1000) * len(tweets)
#   max_count = (1 / 50) * len(tweets)
#   for unigram in list(unigram_counts.keys()):
#     if unigram_counts[unigram] < min_count or unigram_counts[unigram] > max_count:
#       del unigram_counts[unigram]

#   print('unigrams after: {}'.format(len(unigram_counts)))
#   print('NEW Most common: {}'.format(unigram_counts.most_common(10)))

# def remove_freq_infreq_bigrams():
#   print('Skipgram before: {}'.format(len(skipgram_counts)))
#   for x, y in list(skipgram_counts.keys()):
#     if x not in unigram_counts or y not in unigram_counts:
#       del skipgram_counts[(x,y)]

#   print('Skipgram after: {}'.format(len(skipgram_counts)))
#   print('NEW Most common skipgram: {}'.format(skipgram_counts.most_common(10)))


def build_unigram_index_lookup():
  for i, x in enumerate(unigram_counts.keys()):
    x2i[x] = i
    i2x[i] = x


# wwcnt_mat = None
# def word_word_matrix():
#   row = []
#   col = []
#   data = []
#   ii = 0
#   for (tok1, tok2), sg_count in skipgram_counts.items():
#     ii += 1
#     if ii % 200000 == 0:
#       print(f'finished {ii/len(skipgram_counts):.2%} of skipgrams')
#     row.append(tok1)
#     col.append(tok2)
#     data.append(str(sg_count))
#   print(row[0], col[0], data[0])
  
#   data = np.array(data, dtype=float)

#   wwcnt_mat = sparse.csr_matrix((data, (row, col)))
#   print("done")
    

    
def PMI(word1, word2, sx, sxy):
  # You have to store frequencies of individual words and (word, context word) 
  # pairs to compute this. You can compute them beforehand in order to avoid 
  # counting every time when this function is called.
  # Write your code
  a = (skipgram_counts[(word1,word2)] / sxy)
  b = (unigram_counts[word1] / sx)
  c = (unigram_counts[word2] / sx)
  return log(a / b / c)

def build_word_vector(word):
  # Write your code
  sx = sum(unigram_counts.values())
  sxy = sum(skipgram_counts.values())
  data, rows, cols = [], [], []
  pmi_sample = Counter()

  for (x, y), n in skipgram_counts.items():
    if (x == 'mask' and y == word):
      print("I GOTTTTTTT IT", skipgram_counts[('mask', word)])
    if (x != word):
      continue
    rows.append(x2i[x])
    cols.append(x2i[y])
    data.append(PMI(x,y, sx, sxy))
    pmi_sample[(x,y)] = data[-1]

  PMI_matrix = csc_matrix((data, (rows, cols)))
  print('%d non-zero elements' % PMI_matrix.count_nonzero())
  print('Sample PMI values\n', pformat(pmi_sample.most_common()[:10]))
  print(pmi_samples[('mask', word)])


def print_top_dimensions(word_vector, n):
  # print top n dimensions sorted in the order of importance.
  ...

compute_unigrams()
compute_skip_grams()
print()
# remove_freq_infreq_unigrams()
# remove_freq_infreq_bigrams()
print()
build_unigram_index_lookup()
# build_sparce_PMI_matrix()
v1 = build_word_vector('ventilator')

# print_top_dimensions(v1, 20) # print top 20 dimensions along with their weights


finished 0.00% of tweets
finished 15.98% of tweets
finished 31.96% of tweets
finished 47.94% of tweets
finished 63.92% of tweets
finished 79.90% of tweets
finished 95.88% of tweets
Vocab size: 159852
Most common: [('covid', 71281), ('pandemic', 50353), ('’s', 34853), ("'s", 33911), ('covid-19', 33591), ('people', 31850), ('n’t', 31053), ('like', 20837), ('mask', 20107), ('get', 19982)]
finished 0.00% of tweets
finished 15.98% of tweets
finished 31.96% of tweets
finished 47.94% of tweets
finished 63.92% of tweets
finished 79.90% of tweets
finished 95.88% of tweets
Skipgram size: 5327787
Most common skipgram: [(('wear', 'mask'), 11763), (('social', 'distancing'), 6753), (('“', '”'), 6271), (('stay', 'home'), 5871), (('ca', 'n’t'), 4903), (('new', 'cases'), 3304), (('covid', '19'), 3202), (('gon', 'na'), 2875), (('‘', '’'), 2145), (('get', 'covid'), 2044)]


I GOTTTTTTT IT 6
278 non-zero elements
Sample PMI values
 [(('ventilator', 'tessie'), 9.292286118740558),
 (('ventilator', 'attackco

## Problem 1.2: Compute the word similarity between words using Cosine Similarity.

Compute cosine similarity between the following pair of words: 
```
('ventilator', 'covid-19')
('ventilator', 'lockdown')
('ventilator', 'mask')
('ventilator', 'ppe')
```

Outputs of my code are:
```
('ventilator', 'covid-19') 0.17076006036635358
('ventilator', 'lockdown') ...
('ventilator', 'mask') 0.19229601085517933
('ventilator', 'ppe') ...
```


In [ ]:
def cosine_similarity(vector1, vector2):
  # write your code

ventilator = build_word_vector('ventilator')
covid19 = build_word_vector('covid-19')
lockdown = build_word_vector('lockdown')
mask = build_word_vector('mask')
ppe = build_word_vector('ppe')

print(('ventilator', 'covid-19'), cosine_similarity(ventilator, covid19)
print(('ventilator', 'lockdown'), cosine_similarity(ventilator, lockdown))
print(('ventilator', 'mask'), cosine_similarity(ventilator, mask))
print(('ventilator', 'ppe'), cosine_similarity(ventilator, ppe))

## Problem 1.3: What can you tell about these words from the similarities?

1. `ventilator` when compared with `covid-19, lockdown, mask, ppe`
2. `pandemic` when compared with `covid-19, lockdown, mask, ppe`
3. `president` compared with `trump, biden`
4. `trudeau` compared with `trump, biden`



# Let's play with word2vec

First let's load word2vec. I am using [gensim](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html) but feel free to use any libraries or tools.

In [ ]:
from gensim.models import KeyedVectors

EMBEDDING_FILE = '/content/drive/My Drive/tweets/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

## Problem 1.4: Compute the top 5 similar words of `ventilator` using word2vec?

In [ ]:
# Write your code here


# Problem 1.5: Word analogy

If I told you the plural of `car` is `cars`, can you automatically find the plural of `hypothesis` and `man` using word2vec.

Similarly, if I told you a newborn `dog` is called `puppy`, can you automatically find what are the newborn words of `cat` and `sheep` using word2vec?


In [ ]:
# Write your code here


# 2. Topic Models 

The goal of this part is to make you familiar with topic models. You may reuse some of functions you coded for the previous assignments.

## Data Download and Setup

Let us start by downloading the news section of the Brown corpus:

In [ ]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
documents = [brown.words(fileid) for fileid in brown.fileids()]

Let us inspect some of the documents:

In [ ]:
print("The news section of the Brown corpus contains {} documents.".format(len(documents)))
for i in range(3):
  document = documents[i]
  print("Document {} has {} words: {}".format(i, len(document), document))

Finally, let us download a list of stopwords for later.

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')
print(stopwords_list)

## Problem 2.1: Document-Term Matrix

Create a document-term matrix with tf-idf. You should preprocess documents by: 1) lowercasing words, 2) excluding stopwords, and 3) including alphanumeric strings only.

```python
import numpy as np
def create_tfidf_matrix(documents: List[List[str]]) -> (np.array, List[str]):
  # Args:
  #   documents: list of documents, each document being a list of words.
  # Outputs:
  #   tfidf_matrix: np.array of shape (num_documents, vocabulary_size)
  #   vocabulary: a list of terms corresponding to the columns of the matrix.

tfidf_matrix, vocabulary = create_tfidf_matrix(documents)
```

How sparse is this matrix? Calculate the ratio between cells with value 0 and the total number of cells. 


## Problem 2.2: Latent Semantic Analysis

We perform LSA to obtain document embeddings `U` and term embeddings `VT`.

```python
from sklearn.utils.extmath import randomized_svd

U, Sigma, VT = randomized_svd(tfidf_matrix, 
                              n_components=10,
                              n_iter=100,
                              random_state=42)
```

Define a function to find the 5 most relevant terms for each of the 10 latent dimensions (tip: you should make use of VT and the vocabulary).

```python
def extract_salient_words(VT: np.array, 
                  vocabulary: List[str]
                  ) -> salient_words: dict[int, List[str]]:
  # Args:
  #  VT: a numpy array of size (n_components, vocabulary_size)
  #  vocabulary: a list of words of size vocabulary_size
  # Outputs:
  #  salient_words: a dictionary with the latent dimension indices as keys and a list of its 5 most salient words as values

salient_words = extract_salient_words(VT, vocabulary)

for key, value in salient_words:
  print("Concept {}: {}".format(str(key), " ".join(value)))
```

## Problem 2.3: Document Retrieval

Given a text query, view this as a mini document, and compare it to your documents in the low-dimensional space.

First, we need to map a query into a pseudo-document embedding.
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/ed5d0397ee6b44f72f77743029d3943932118fa2" alt="Query" height="35"/>

Then, you will need to implement a function to calculate the cosine similarity between this embedded query and all the document embeddings.

Retrieve the indices of the top-3 documents with the highest cosine similarity with the following queries:


```python
query1 = ['T.', 'V.', 'Barker', 'developed', 'the', 'classification-angle', 'system']
query2 = ['imitation', 'vs.', 'formalism' 'in', 'philosophical', 'debates']
query3 = ['Krim', 'attended', 'the', 'University', 'of', 'North', 'Carolina', 'to', 'follow', 'Thomas', 'Wolfe']
```



## Problem 2.4: Document Clustering

```python
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

num_clusters = 10
km = KMeans(n_clusters=num_clusters)
document_embeddings = U * Sigma
km.fit(document_embeddings)
clusters = km.labels_.tolist()
print(clusters)
```

Let us now plot the document embeddings and their clusters:

```python
import umap
embedding = umap.UMAP(n_neighbors=100, min_dist=0.5, random_state=42).fit_transform(document_embeddings)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], c=clusters, s=20, edgecolor='none')
plt.show()
```

What are the differences you observe by using a different number of `n_components` in LSA or `n_clusters` in K-Means?

## Problem 2.5 Latent Dirichlet Allocation

Run LDA on `documents` using `sklearn` (find the documentation at this [link](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#sklearn.decomposition.LatentDirichletAllocation))

Make sure to specify `random_state=42` for replicability. 

What are the topics allocated to each word of document number 13? 

In [ ]:
print(documents[13])

In [ ]:
# Write your code here.
    
